In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317


In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Any, m_v::Missing) = missing
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Missing, m_v::Any) = missing

@rule typeof(+)(:in1, Marginalisation) (m_out::Missing, m_in2::Any) = missing
@rule typeof(+)(:in1, Marginalisation) (m_out::Any, m_in2::Missing) = missing

In [4]:
P = 1.0

n = 500
data = convert(Vector{Union{Float64, Missing}}, collect(1:n) + rand(Normal(0.0, sqrt(P)), n));

for index in map(d -> rem(abs(d), n), rand(Int, Int(n / 2)))
    data[index] = missing
end

In [5]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, P);

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [6]:
x0_prior = NormalMeanVariance(0.0, 1000.0)
res = inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9288876393326935, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=1.9288876393326935, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=2.9288876393326935, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=3.9288876393326935, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=4.928887639332693, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=5.928887639332691, v=0.0032894628636090033))
 Marginal(NormalMeanVariance{Float64}(μ=6.928887639332693, v=0.0032894628636090037))
 Marginal(NormalMeanVariance{Float64}(μ=7.928887639332693, v=0.003289462863609004))
 Marginal(NormalMeanVariance{Float64}(μ=8.928887639332693, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=9.928887639332691, v=0.0032894628636090046))
 Marginal(NormalMeanVariance{Float64}(μ=10.928887639332691, v=0.003289462863609004))
 Marginal(NormalMeanVariance{Flo